### set the base params

In [1]:
root = '/home/bob/projects/Web3HackerNetwork'
dataset_dir = root + "/data/github/2022-04-07-marine-search"
stats_dir = dataset_dir + "/projects"

### define a method that finds all files with the same name

In [2]:
import os
def find_all(name, path):
    result = []
    for root, dirs, files in os.walk(path):
        if name in files:
            result.append(os.path.join(root, name))
    return result

### find all the commi stat JSON files

In [3]:
commit_files = find_all('commit-stat.log.json', stats_dir)

### load all the files

In [4]:
import json

all_logs = []
for file in commit_files:
    in_stats = json.loads(open(file, 'r').read());
    all_logs.append(in_stats)
print(all_logs[0][0])

{'commit': 'fb528905fb6448e2b502f7de8fba877a2a995f6d', 'Author': 'tchataigner <tom.chataigner@yahoo.fr>', 'Date': 'Sun Aug 15 20', 'fileTypes': {'js': {'textLineCount': 26, 'binByteCount': 0, 'inserts': 17, 'deletes': 9, 'occurrences': 2}}, 'files': '2', 'insertions(+)': '17', 'deletions(-)\n': '9'}


In [5]:
js_commits = []
rust_commits = []
all_commits = []

js_types = ['js', 'jsx', 'ts', 'tsx']
rust_types = ['rs', 'toml']

for log in all_logs:
    #lines.append("\n")
    for commit in log:
        #lines.append(str(commit['commit']))
        #print(commit['fileTypes'])
        for ft in commit['fileTypes']:
            all_commits.append(commit)
            if ft in js_types:
                js_commits.append(commit)
            elif ft in rust_types:
                rust_commits.append(commit)

### method to extract statistics from the commit data

In [23]:
js_types = ['js', 'jsx', 'ts', 'tsx']
rust_types = ['rs', 'toml']

def percent(numerator, denominator):
    if (numerator == 0): return 0
    truncated = int(100 * 100 * numerator / denominator)
    return truncated / 100

def extract_stats(commit):
    if 'files' in commit:
        num_files = commit['files']
    else:
        num_files = commit['file']
    typeDict = commit['fileTypes']
    typeArray = [{'fileType': key, 'stats': typeDict[key]} for key in typeDict.keys()]
    textTypeArray = [typeEntry for typeEntry in typeArray if typeEntry['stats']['textLineCount'] > 0]
    binTypeArray = [typeEntry for typeEntry in typeArray if typeEntry['stats']['binByteCount'] > 0]
    jsTypeArray = [typeEntry for typeEntry in typeArray if typeEntry['fileType'] in js_types]
    rsTypeArray = [typeEntry for typeEntry in typeArray if typeEntry['fileType'] in rust_types]
    
    statDict = {}
    statDict['commit'] = commit['commit']
    statDict['totalFiles'] = int(num_files)
    statDict['binFiles'] = sum([typeEntry['stats']['occurrences'] for typeEntry in binTypeArray])
    statDict['textFiles'] = sum([typeEntry['stats']['occurrences'] for typeEntry in textTypeArray])
    statDict['rustFiles'] = sum([typeEntry['stats']['occurrences'] for typeEntry in rsTypeArray])
    statDict['jsFiles'] = sum([typeEntry['stats']['occurrences'] for typeEntry in jsTypeArray])
    statDict['binBytes'] = sum([typeEntry['stats']['binByteCount'] for typeEntry in binTypeArray])
    statDict['textLines'] = sum([typeEntry['stats']['textLineCount'] for typeEntry in textTypeArray])
    statDict['rustLines'] = sum([typeEntry['stats']['textLineCount'] for typeEntry in rsTypeArray])
    statDict['jsLines'] = sum([typeEntry['stats']['textLineCount'] for typeEntry in jsTypeArray])
    statDict['totalBytes'] = statDict['textLines'] * 50 + statDict['binBytes']
    
    statDict['pctRustFiles'] = percent(statDict['rustFiles'], statDict['totalFiles'])
    statDict['pctJsFiles'] = percent(statDict['jsFiles'], statDict['totalFiles'])
    statDict['pctRustLines'] = percent(statDict['rustLines'], statDict['textLines'])
    statDict['pctJsLines'] = percent(statDict['jsLines'], statDict['textLines'])
    statDict['pctRustBytes'] = percent(statDict['rustLines'] * 50, statDict['totalBytes'])
    statDict['pctJsBytes'] = percent(statDict['jsLines'] * 50, statDict['totalBytes'])
    statDict['pctBinBytes'] = percent(statDict['binBytes'], statDict['totalBytes'])
    statDict['pctTextBytes'] = percent(statDict['textLines'] * 50, statDict['totalBytes'])
    return statDict
    # fileType['textLineCount'] for fileType in commit['fileTypes']]

In [32]:
all_stats = []
commit_map = {}
stat_map = {}

for commit in all_commits:
    commit_id = commit['commit']
    statDict = extract_stats(commit)
    all_stats.append(statDict)
    commit_map[commit_id] = commit
    stat_map[commit_id] = statDict

feature_stat_keys = [
    'pctRustFiles', 'pctJsFiles',
    'pctRustLines', 'pctJsLines',
    'pctRustBytes', 'pctJsBytes',
    'pctBinBytes', 'pctTextBytes'
]

feature_stat_map = {}
for commit_id in stat_map.keys():
    feature_stats = {}
    feature_stat_map[commit_id] = feature_stats
    for stat_key in feature_stat_keys:
        feature_stats[stat_key] = stat_map[commit_id][stat_key]

count = 0
for commit_id in stat_map.keys():
    count += 1
    if count > 50: break
    # print(commit_map[commit_id])
    # print(stat_map[commit_id])
    print(feature_stat_map[commit_id])
    # print()



{'pctRustFiles': 0, 'pctJsFiles': 100.0, 'pctRustLines': 0, 'pctJsLines': 100.0, 'pctRustBytes': 0, 'pctJsBytes': 100.0, 'pctBinBytes': 0, 'pctTextBytes': 100.0}
{'pctRustFiles': 0, 'pctJsFiles': 0, 'pctRustLines': 0, 'pctJsLines': 0, 'pctRustBytes': 0, 'pctJsBytes': 0, 'pctBinBytes': 0, 'pctTextBytes': 100.0}
{'pctRustFiles': 0, 'pctJsFiles': 0, 'pctRustLines': 0, 'pctJsLines': 0, 'pctRustBytes': 0, 'pctJsBytes': 0, 'pctBinBytes': 0, 'pctTextBytes': 100.0}
{'pctRustFiles': 0, 'pctJsFiles': 0, 'pctRustLines': 0, 'pctJsLines': 0, 'pctRustBytes': 0, 'pctJsBytes': 0, 'pctBinBytes': 0, 'pctTextBytes': 100.0}
{'pctRustFiles': 0, 'pctJsFiles': 0, 'pctRustLines': 0, 'pctJsLines': 0, 'pctRustBytes': 0, 'pctJsBytes': 0, 'pctBinBytes': 0, 'pctTextBytes': 100.0}
{'pctRustFiles': 0, 'pctJsFiles': 100.0, 'pctRustLines': 0, 'pctJsLines': 100.0, 'pctRustBytes': 0, 'pctJsBytes': 100.0, 'pctBinBytes': 0, 'pctTextBytes': 100.0}
{'pctRustFiles': 0, 'pctJsFiles': 0, 'pctRustLines': 0, 'pctJsLines': 0, 'pc